In [8]:
from base64 import b64decode
from collections import defaultdict
from os import environ
from pprint import pprint

from dnslib import DNSRecord
from yaml import safe_load

from survey import RipeClient

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [24]:
# as-is
main_measurement_id = "51339831"
# with #37
pr_measurement_id = "51339896"

In [12]:
#import logging
#logging.basicConfig(level=logging.DEBUG)

In [13]:
ripe = RipeClient()

In [35]:
main_results = ripe.measurement_results(main_measurement_id)
pr_results = ripe.measurement_results(pr_measurement_id)

In [36]:
ips = {}
with open('/Users/ross/octodns/octodns-ns1/config/ordering.com.yaml') as fh:
    data = safe_load(fh)
    for name, pool in data['cdn-dev'][0]['dynamic']['pools'].items():
        for value in pool['values']:
            ips[value['value']] = name
pprint(ips)

{'103.231.91.78': 'ocean',
 '137.74.0.47': 'eu_E',
 '138.199.57.66': 'na',
 '138.199.57.67': 'na',
 '146.59.70.106': 'eu_E',
 '185.172.148.128': 'eu',
 '185.172.148.132': 'eu',
 '185.172.149.128': 'eu_nl',
 '185.172.149.132': 'eu_nl',
 '62.210.90.237': 'eu_fr',
 '62.210.90.238': 'eu_fr',
 '68.70.192.128': 'eu_uk',
 '82.102.17.19': 'eu_SW',
 '87.249.130.6': 'sa',
 '89.187.160.25': 'apac',
 '89.187.160.31': 'apac'}


In [37]:
main_answers = defaultdict(set)
for result in main_results:
    probe_id = result['prb_id']
    probe_ids.append(probe_id)
    probe_location = ripe.probe(probe_id)['country_code']
    try:
        abuf = result['result']['abuf']
    except KeyError:
        continue
    for rr in DNSRecord.parse(b64decode(abuf)).rr:
        answer = str(rr.rdata)
        try:
            pool = ips[answer]
        except KeyError:
            pool = 'unknown'
        main_answers[probe_location].add(pool)

In [38]:
pr_answers = defaultdict(set)
for result in pr_results:
    probe_id = result['prb_id']
    probe_ids.append(probe_id)
    probe_location = ripe.probe(probe_id)['country_code']
    try:
        abuf = result['result']['abuf']
    except KeyError:
        continue
    for rr in DNSRecord.parse(b64decode(abuf)).rr:
        answer = str(rr.rdata)
        try:
            pool = ips[answer]
        except KeyError:
            pool = 'unknown'
        pr_answers[probe_location].add(pool)

In [39]:
for loc, main_pool in main_answers.items():
    pr_pool = pr_answers[loc]
    if main_pool != pr_pool:
        print(loc)
        print(f'  {",".join(main_pool)}')
        print(f'  {",".join(pr_pool)}')

IR
  apac
  ocean,apac
AU
  apac
  ocean
NZ
  apac
  ocean
